In [ ]:
import os
import sys
import math
import numpy as np
import matplotlib.pyplot as plt

# Add path of project root
sys.path.append(os.path.abspath('..'))

# Import utils module.
from utils import future_value_daily_compound



In [ ]:


def simulate_credit_card(principal, apr, 
                         total_days,            # simulation length (days)
                         payment_interval=30,       # Payment interval (days)
                         min_payment_fixed=25,      # minimum payments floor
                         payment_perc_coef = 0.05   # Minimum payment rule coef.
                    ):
    """Simulate credit card balance and payments over time."""
    # Initialize
    daily_rate = apr / 365          # Daily interest rate
    days = np.arange(0, total_days + 1)
    balance = np.zeros_like(days, dtype=float)
    payments = np.zeros_like(days, dtype=float)
    daily_interest = np.zeros_like(days, dtype=float)
    cumulative_interest = np.zeros_like(days, dtype=float)

    balance[0] =  principal    

    # Simulate day by day
    for i in range(1, len(days)):
        # Interest accrued
        daily_interest[i] = balance[i-1] * daily_rate
        cumulative_interest[i] = cumulative_interest[i - 1] + daily_interest[i]
        
        # Apply daily interest
        balance[i] = balance[i-1] + daily_interest[i]

        # Make a payment every `payment interval` days
        if i % payment_interval == 0:
            # Interest from last interval
            interest_period = cumulative_interest[i] - cumulative_interest[i - payment_interval]

            # Minimum payment rule
            percent_based = payment_perc_coef * balance[i] + interest_period
            payment = max(min_payment_fixed, percent_based)

            # Cap payment
            payment = min(payment, balance[i])
            balance[i] -= payment
            payments[i] += payment
    
    return days, balance, payments, cumulative_interest

def simulate_credit_card_fixed_payment(principal, apr, 
                         total_days,            # simulation length (days)
                         payment_interval=30,       # Payment interval (days)
                         min_payment_fixed=25,      # minimum payments floor
                         fv_margin_weight=0.71         # amount of calculated fv to determine fixed payment
                    ):
    """Simulate credit card balance and payments over time. (Fixed payment method)"""
    # Initialize
    daily_rate = apr / 365          # Daily interest rate
    days = np.arange(0, total_days + 1)
    balance = np.zeros_like(days, dtype=float)
    payments = np.zeros_like(days, dtype=float)
    daily_interest = np.zeros_like(days, dtype=float)
    cumulative_interest = np.zeros_like(days, dtype=float)

    balance[0] =  principal

    # Calculate optimal fixed payments.
    fv = future_value_daily_compound(principal, apr, total_days)
    optimal_fixed_payment = fv_margin_weight * fv / math.ceil(total_days / payment_interval)

    # Simulate day by day
    for i in range(1, len(days)):
        # Interest accrued
        daily_interest[i] = balance[i-1] * daily_rate
        cumulative_interest[i] = cumulative_interest[i - 1] + daily_interest[i]
        
        # Apply daily interest
        balance[i] = balance[i-1] + daily_interest[i]
        
        # Make a payment every `payment interval` days
        if i % payment_interval == 0:
            # Fixed payment rule
            payment = max(min_payment_fixed, optimal_fixed_payment)

            # Cap payment
            payment = min(payment, balance[i])
            balance[i] -= payment
            payments[i] += payment
    
    return days, balance, payments, cumulative_interest

"""MAIN CODE"""
# parameters
simulate_num_days = 365*4

principal1 = 1000.0             # initial balance
apr1 = 0.24                   # Annual Perecentage Rate (APR)

principal2 = 4300.0
apr2 = 0.18

# Simulate multiple accounts

days1, balance1, payments1, interest1 = simulate_credit_card(principal1, apr1, simulate_num_days)
days2, balance2, payments2, interest2 = simulate_credit_card(principal2, apr2, simulate_num_days)

# Combine results
total_principal = principal1 + principal2
total_balance = balance1 + balance2
total_payments = payments1 + payments2
total_interest = interest1 + interest2

# Balance Plot
plt.figure(figsize=(10,6))
plt.plot(days1, total_balance, label="Balance with Daily Compounding + Payments", color='darkorange')
plt.axhline(total_principal, color='gray', linestyle='--', label="Initial Principal")
plt.axhline(total_balance[-1], color='red', linestyle='--', label="Future Principal")
plt.xlabel("Days")
plt.ylabel("Remaining Balance ($)")
plt.title("Credit Card Balance With Daily Compound Interest and Minimum Payments")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Payment Stem Plot
payment_days = days1[total_payments > 0]
payment_values = total_payments[total_payments > 0]

plt.figure(figsize=(10,4))
plt.stem(payment_days, payment_values, linefmt='gray', markerfmt='ro', basefmt=" ")
plt.xlabel("Day")
#plt.xticks(payment_days)
plt.ylabel("Payment Amount ($)")
plt.title("Payment Amounts at Each Interval")
plt.grid(True)
plt.tight_layout()
plt.show()

# Cumulative Interest Plot
plt.figure(figsize=(10,6))
plt.plot(days1, total_interest, label="Total Interest", color="purple")
plt.xlabel("Day")
plt.ylabel("Cummulative Interest ($)")
plt.title("Cumulative Interst Accrued Over Time")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Print
print(f"Initial Principal Value at Day 0: ${total_balance[0]:.2f}")
print(f"Final Principal Value at Day {days1[-1]}: ${total_balance[-1]:.2f}")
print(f"Cumulative Interest at Day {days1[-1]}: ${total_interest[-1]:.2f}")

zero_indices = np.where(total_balance==0)[0] #get index when balance equal $0)
if zero_indices.size > 0: # Print out Day when balance equals $0
    print(f"***Balance paid off at Day {days1[zero_indices[0]]}.***")

In [ ]:
# Fixed Payment Method

# Simulate multiple accounts

days1, balance1, payments1, interest1 = simulate_credit_card_fixed_payment(principal1, apr1, simulate_num_days)
days2, balance2, payments2, interest2 = simulate_credit_card_fixed_payment(principal2, apr2, simulate_num_days)

# Combine results
total_principal2 = principal1 + principal2
total_balance2 = balance1 + balance2
total_payments2 = payments1 + payments2
total_interest2 = interest1 + interest2

# Balance Plot
plt.figure(figsize=(10,6))
plt.plot(days1, total_balance2, label="Balance with Daily Compounding + Payments", color='darkorange')
plt.axhline(total_principal2, color='gray', linestyle='--', label="Initial Principal")
plt.axhline(total_balance2[-1], color='red', linestyle='--', label="Future Principal")
plt.xlabel("Days")
plt.ylabel("Remaining Balance ($)")
plt.title("Credit Card Balance With Daily Compound Interest and Minimum Payments")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Payment Stem Plot
payment_days = days1[total_payments2 > 0]
payment_values = total_payments2[total_payments2 > 0]

plt.figure(figsize=(10,4))
plt.stem(payment_days, payment_values, linefmt='gray', markerfmt='ro', basefmt=" ")
plt.xlabel("Day")
#plt.xticks(payment_days)
plt.ylabel("Payment Amount ($)")
plt.title("Payment Amounts at Each Interval")
plt.grid(True)
plt.tight_layout()
plt.show()

# Cumulative Interest Plot
plt.figure(figsize=(10,6))
plt.plot(days1, total_interest2, label="Total Interest", color="purple")
plt.xlabel("Day")
plt.ylabel("Cummulative Interest ($)")
plt.title("Cumulative Interst Accrued Over Time")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Print
print(f"Initial Principal Value at Day 0: ${total_balance2[0]:.2f}")
print(f"Final Principal Value at Day {days1[-1]}: ${total_balance2[-1]:.2f}")
print(f"Cumulative Interest at Day {days1[-1]}: ${total_interest2[-1]:.2f}")

zero_indices = np.where(total_balance2==0)[0] #get index when balance equal $0)
if zero_indices.size > 0: # Print out Day when balance equals $0
    print(f"***Balance paid off at Day {days1[zero_indices[0]]}.***")

In [ ]:
"""Overlay Plots Together (Minimum Payment Rule vs Fixed Payment)"""
# Balance Plot
plt.figure(figsize=(10,6))
plt.plot(days1, total_balance, label="Minimum Payment Rule", color='darkorange')
plt.plot(days1, total_balance2, label="Fixed Payment", color='green')
#plt.axhline(total_principal, color='gray', linestyle='--', label="Initial Principal")
#plt.axhline(total_balance[-1], color='red', linestyle='--', label="Future Principal")
plt.xlabel("Days")
plt.ylabel("Remaining Balance ($)")
plt.title("Credit Card Balance Comparison (Minimum Rule vs Fixed)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Cumulative Interest Plot
plt.figure(figsize=(10,6))
plt.plot(days1, total_interest, label="Total Interest Minimum Rule", color="purple")
plt.plot(days1, total_interest2, label="Total Interest Fixed", color="magenta")
plt.xlabel("Day")
plt.ylabel("Cummulative Interest ($)")
plt.title("Cumulative Interst Accrued Over Time (Minimum Rule vs Fixed)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()
